In [2]:
# 放映
# jupyter nbconvert base.ipynb --to slides --post serve

# 什么是分布式锁

分布式锁： 在分布式系统中，解决共享资源的串行话访问问题，
解决了单机在分布式系统中失效的问题。

# 分布式事务锁应用场景

In [ ]:
## 场景01

![alt text](./imgs/分布式事务锁应用场景.png "Title")

In [ ]:
## 场景02

![alt text](./imgs/cj02.png  "Title")

In [ ]:
## 场景03

- 贵州 ---> 上海 ---> 洛杉矶

![alt text](./imgs/cj03.png  "Title")

In [ ]:
# 分布式事务设计原理

In [ ]:
## 简要模型

In [ ]:
### 分布式锁是什么
- 没有操作系统层面的机制解决多机的串行化
- 某个节点持有“锁”，无法阻止其他节点的运行
- 业务约定好，使用共享存储

In [ ]:
## 分布式锁业务模型
- 获取锁： 向公共存储中写入某个值
    - 成功： 要写入的值不存在
    - 失败： 要写入的值存在
- 释放锁： 删除获取锁时写入的值

In [ ]:
![alt text](./imgs/分布式锁业务模型.png "Title")

## 需求分析：
功能需求
 - 提供获取锁/释放锁操作接口
 - 冗余部署的多个模块串行化访问临界资源

业务需求
 - 锁的一致性需求

性能需求
 - 服务并发量及响应延时

In [ ]:
## 分布式锁深入思考

### 客户端

- 逻辑控制
- 确保锁正常释放

### 客户端
- 逻辑控制
  - 业务正确使用
- 确保锁正常释放
  - 为锁设置有效期
  - 有效期长短如何设置

释放锁如何校验

![alt text](2.png "Title")

SET if Not eXistsSET if Not eXists

![alt text](分布式01.png "Title")

1. 程序处理业务逻辑异常，没及时释放锁
2. 进程挂了，没机会释放锁

这个客户端就会一直占用这个锁，而其它客户端就「永远」拿不到这把锁了。


![alt text](./imgs/fbssw死锁.png "Title")

- 引入TTL 过期时间

![alt text](./imgs/fbsswsTTL.png  "Title")

- 引入TTL 过期时间 出现的问题

![alt text](./imgs/fbsswsttlerror.png "Title")

- 锁过期时间如何评估
- 锁被别人释放

- 续租模式 解决【锁过期时间如何评估】

![alt text](./imgs/fbsswxz.png "Title")

- 设置锁值 解决【锁被别人释放】

![alt text](./imgs/fbsswspt.png "Title")

- Lock： 添加key，设置有效期 【加锁操作的原子性】
- unLock: 删除Key
- 有效期维护： 后台续租 【续租和释放锁安全性】

![alt text](分布式03.png "Title")

工作时序：
- 竞争锁
- 释放锁
- 上报监控

![alt text](分布式04.png "Title")

# 服务端

zookeeper
redis
etcd

<h1 style="color:red">选择一个合适的存储</h1>

![alt text](分布式05.png "Title")

<a href='https://www.ruanyifeng.com/blog/2018/07/cap.html'>CAP</a>

- Consistency 数据一致性
- Availability 服务可用性
- Partition tolerance 分区容错


![alt text](分布式06.png "Title")

redis存储 【AP】

- 主从切换数据丢失
- 部分锁失效

etcd 【CP】
- 多副本存储，数据不丢失
- 选主过程不可用


zk 【CP】
- 多副本存储，数据不丢失
- 选主过程不可用


![alt text](分布式07.png "Title")

![alt text](分布式08.png "Title")

In [ ]:
谁持有锁，是否可以续租，对锁的清理，完全交由业务，误操作风险很大，所以CAS的机制尤为重要！！！

In [ ]:
# 任期+编号

ZooKeeper是一个分布式的，开放源码的分布式应用程序协调服务，是Google的Chubby一个开源的实现，是Hadoop和Hbase的重要组件。它是一个为分布式应用提供一致性服务的软件，提供的功能包括：配置维护、域名服务、分布式同步、组服务等。

In [ ]:
# 为什么一般的节点是奇数个

N+1 可用
8个节点 N+1=5 ， 丢失数 8-5 = 3
7个节点 N+1=4 ， 丢失数 7-4 = 3

所以答案是节省成本

ZK的选举是有 任期+编号实现，  不会出现同票情况。


# 红锁
1. 客户端先获取「当前时间戳T1」
2. 客户端依次向这 5 个 Redis 实例发起加锁请求（用前面讲到的 SET 命令），且每个请求会设置超时时间（毫秒级，要远小于锁的有效时间），如果某一个实例加锁失败（包括网络超时、锁被其它人持有等各种异常情况），就立即向下一个 Redis 实例申请加锁
3. 如果客户端从 >=3 个（大多数）以上 Redis 实例加锁成功，则再次获取「当前时间戳T2」，如果 T2 - T1 < 锁的过期时间，此时，认为客户端加锁成功，否则认为加锁失败
4. 加锁成功，去操作共享资源（例如修改 MySQL 某一行，或发起一个 API 请求）
5. 加锁失败，向「全部节点」发起释放锁请求（前面讲到的 Lua 脚本释放锁）

In [ ]:
前面也分析了，Redlock 只有建立在「时钟正确」的前提下，才能正常工作，如果你可以保证这个前提，那么可以拿来使用。

In [ ]:
# Zookeeper 的锁
1. 客户端 1 和 2 都尝试创建「临时节点」，例如 /lock
2. 假设客户端 1 先到达，则加锁成功，客户端 2 加锁失败
3. 客户端 1 操作共享资源
4. 客户端 1 删除 /lock 节点，释放锁【客户端 1 异常崩溃了，那么这个临时节点会自动删除】

![alt text](zklock.png  "Title")